# 豆瓣电影---按分类爬取
我突然想看下有什么电影可以看。由于我偏爱剧情类电影，因此我用Python爬虫来爬取剧情类型的电影。




## 一、单个页面分析及爬取

In [2]:
import requests
from lxml import etree
import json
import pandas as pd

下面的url是通过chrome的“检查”得到的。

In [4]:
url = "https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=20&genres=%E5%89%A7%E6%83%85&countries=%E7%BE%8E%E5%9B%BD"
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"}
data = requests.get(url,headers = headers).text

Python中json.loads和json.dumps对比：
- json.dumps 将Python中的字典转换为字符串  
- json.loads 将字符串转换为Python中的字典

In [7]:
dicts = json.loads(data)
dicts

{'data': [{'directors': ['朱塞佩·托纳多雷'],
   'rate': '8.8',
   'cover_x': 1201,
   'star': '45',
   'title': '西西里的美丽传说',
   'url': 'https://movie.douban.com/subject/1292402/',
   'casts': ['莫妮卡·贝鲁奇', '朱塞佩·苏尔法罗', '埃丽萨·莫鲁奇'],
   'cover': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p792400696.jpg',
   'id': '1292402',
   'cover_y': 1704},
  {'directors': ['弗朗西斯·福特·科波拉'],
   'rate': '9.2',
   'cover_x': 800,
   'star': '45',
   'title': '教父',
   'url': 'https://movie.douban.com/subject/1291841/',
   'casts': ['马龙·白兰度', '阿尔·帕西诺', '詹姆斯·肯恩', '理查德·卡斯特尔诺', '罗伯特·杜瓦尔'],
   'cover': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2190556185.jpg',
   'id': '1291841',
   'cover_y': 1200},
  {'directors': ['乔纳森·戴米'],
   'rate': '8.7',
   'cover_x': 2489,
   'star': '45',
   'title': '沉默的羔羊',
   'url': 'https://movie.douban.com/subject/1293544/',
   'casts': ['朱迪·福斯特', '安东尼·霍普金斯', '斯科特·格伦', '安东尼·希尔德', '布鲁克·史密斯'],
   'cover': 'https://img1.doubanio.com/view/photo/s_ratio_poster

In [14]:
df = pd.DataFrame(dicts["data"])
df.head()

,casts,cover,cover_x,cover_y,directors,id,rate,star,title,url
0,"[莫妮卡·贝鲁奇, 朱塞佩·苏尔法罗, 埃丽萨·莫鲁奇]",https://img3.doubanio.com/view/photo/s_ratio_p...,1201,1704,[朱塞佩·托纳多雷],1292402,8.8,45,西西里的美丽传说,https://movie.douban.com/subject/1292402/
1,"[马龙·白兰度, 阿尔·帕西诺, 詹姆斯·肯恩, 理查德·卡斯特尔诺, 罗伯特·杜瓦尔]",https://img3.doubanio.com/view/photo/s_ratio_p...,800,1200,[弗朗西斯·福特·科波拉],1291841,9.2,45,教父,https://movie.douban.com/subject/1291841/
2,"[朱迪·福斯特, 安东尼·霍普金斯, 斯科特·格伦, 安东尼·希尔德, 布鲁克·史密斯]",https://img1.doubanio.com/view/photo/s_ratio_p...,2489,3686,[乔纳森·戴米],1293544,8.7,45,沉默的羔羊,https://movie.douban.com/subject/1293544/
3,"[莱昂纳多·迪卡普里奥, 马克·鲁弗洛, 本·金斯利, 马克斯·冯·叙多夫, 米歇尔·威廉姆斯]",https://img1.doubanio.com/view/photo/s_ratio_p...,2480,3543,[马丁·斯科塞斯],2334904,8.7,45,禁闭岛,https://movie.douban.com/subject/2334904/
4,"[克里斯蒂安·贝尔, 希斯·莱杰, 艾伦·艾克哈特, 迈克尔·凯恩, 玛吉·吉伦哈尔]",https://img3.doubanio.com/view/photo/s_ratio_p...,4050,6000,[克里斯托弗·诺兰],1851857,9.1,45,蝙蝠侠：黑暗骑士,https://movie.douban.com/subject/1851857/


删除DataFrame中不需要的列。

In [15]:
df.drop("cover_x",axis = 1,inplace = True)
df.drop("cover_y",axis = 1,inplace = True)
df.head()

,casts,cover,directors,id,rate,star,title,url
0,"[莫妮卡·贝鲁奇, 朱塞佩·苏尔法罗, 埃丽萨·莫鲁奇]",https://img3.doubanio.com/view/photo/s_ratio_p...,[朱塞佩·托纳多雷],1292402,8.8,45,西西里的美丽传说,https://movie.douban.com/subject/1292402/
1,"[马龙·白兰度, 阿尔·帕西诺, 詹姆斯·肯恩, 理查德·卡斯特尔诺, 罗伯特·杜瓦尔]",https://img3.doubanio.com/view/photo/s_ratio_p...,[弗朗西斯·福特·科波拉],1291841,9.2,45,教父,https://movie.douban.com/subject/1291841/
2,"[朱迪·福斯特, 安东尼·霍普金斯, 斯科特·格伦, 安东尼·希尔德, 布鲁克·史密斯]",https://img1.doubanio.com/view/photo/s_ratio_p...,[乔纳森·戴米],1293544,8.7,45,沉默的羔羊,https://movie.douban.com/subject/1293544/
3,"[莱昂纳多·迪卡普里奥, 马克·鲁弗洛, 本·金斯利, 马克斯·冯·叙多夫, 米歇尔·威廉姆斯]",https://img1.doubanio.com/view/photo/s_ratio_p...,[马丁·斯科塞斯],2334904,8.7,45,禁闭岛,https://movie.douban.com/subject/2334904/
4,"[克里斯蒂安·贝尔, 希斯·莱杰, 艾伦·艾克哈特, 迈克尔·凯恩, 玛吉·吉伦哈尔]",https://img3.doubanio.com/view/photo/s_ratio_p...,[克里斯托弗·诺兰],1851857,9.1,45,蝙蝠侠：黑暗骑士,https://movie.douban.com/subject/1851857/


## 二、一次性爬取多个url

In [1]:
import requests
from lxml import etree
import json
import pandas as pd

In [2]:
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"}

In [3]:
for i in range(0,481,20):
    url = "https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start={页面}&genres=%E5%89%A7%E6%83%85&countries=%E7%BE%8E%E5%9B%BD".format(页面 = i)
    data = requests.get(url,headers = headers).text
    dicts = json.loads(data)
    df = pd.DataFrame(dicts["data"])
    if i == 0:
        total_df = df
    else:
        total_df = pd.concat([total_df,df],axis = 0)

In [4]:
total_df

,casts,cover,cover_x,cover_y,directors,id,rate,star,title,url
0,"[蒂姆·罗宾斯, 摩根·弗里曼, 鲍勃·冈顿, 威廉姆·赛德勒, 克兰西·布朗]",https://img3.doubanio.com/view/photo/s_ratio_p...,2000,2963,[弗兰克·德拉邦特],1292052,9.6,50,肖申克的救赎,https://movie.douban.com/subject/1292052/
1,"[莱昂纳多·迪卡普里奥, 约瑟夫·高登-莱维特, 艾伦·佩吉, 汤姆·哈迪, 渡边谦]",https://img3.doubanio.com/view/photo/s_ratio_p...,2229,3300,[克里斯托弗·诺兰],3541415,9.3,45,盗梦空间,https://movie.douban.com/subject/3541415/
2,"[汤姆·汉克斯, 罗宾·怀特, 加里·西尼斯, 麦凯尔泰·威廉逊, 莎莉·菲尔德]",https://img1.doubanio.com/view/photo/s_ratio_p...,721,1035,[罗伯特·泽米吉斯],1292720,9.4,45,阿甘正传,https://movie.douban.com/subject/1292720/
3,"[莱昂纳多·迪卡普里奥, 凯特·温丝莱特, 比利·赞恩, 凯西·贝茨, 弗兰西丝·费舍]",https://img3.doubanio.com/view/photo/s_ratio_p...,2015,3000,[詹姆斯·卡梅隆],1292722,9.3,45,泰坦尼克号,https://movie.douban.com/subject/1292722/
4,"[玛德琳·卡罗尔, 卡兰·麦克奥利菲, 瑞贝卡·德·莫妮, 安东尼·爱德华兹, 约翰·马奥尼]",https://img3.doubanio.com/view/photo/s_ratio_p...,1076,1600,[罗伯·莱纳],3319755,8.9,45,怦然心动,https://movie.douban.com/subject/3319755/
5,"[威尔·史密斯, 贾登·史密斯, 坦迪·牛顿, 布莱恩·豪威 , 詹姆斯·凯伦]",https://img3.doubanio.com/view/photo/s_ratio_p...,1000,1500,[加布里埃莱·穆奇诺],1849031,9.0,45,当幸福来敲门,https://movie.douban.com/subject/1849031/
6,"[苏拉·沙玛, 伊尔凡·可汗, 拉菲·斯波, 阿迪勒·侯赛因, 塔布]",https://img3.doubanio.com/view/photo/s_ratio_p...,3375,5000,[李安],1929463,9.0,45,少年派的奇幻漂流,https://movie.douban.com/subject/1929463/
7,"[理查·基尔, 萨拉·罗默尔, 琼·艾伦, 罗比·萨布莱特, 艾瑞克·阿瓦利]",https://img3.doubanio.com/view/photo/s_ratio_p...,1433,2028,[拉斯·霍尔斯道姆],3011091,9.3,45,忠犬八公的故事,https://movie.douban.com/subject/3011091/
8,"[约翰尼·德普, 薇诺娜·瑞德, 黛安·韦斯特, 安东尼·迈克尔·豪尔, 凯西·贝克]",https://img1.doubanio.com/view/photo/s_ratio_p...,498,755,[蒂姆·波顿],1292370,8.7,45,剪刀手爱德华,https://movie.douban.com/subject/1292370/
9,"[金·凯瑞, 劳拉·琳妮, 艾德·哈里斯, 诺亚·艾默里奇, 娜塔莎·麦克艾霍恩]",https://img3.doubanio.com/view/photo/s_ratio_p...,997,1500,[彼得·威尔],1292064,9.1,45,楚门的世界,https://movie.douban.com/subject/1292064/


写入csv文件  
第一个参数是说把dataframe写入到D盘下的a.csv文件中，参数sep表示字段之间用’,’分隔，header表示是否需要头部，index表示是否需要行号。

In [5]:
total_df.to_csv('movie-0.csv', sep = ',', header = True, index = False)